In [1]:
import os
import logging
import pandas as pd
logger = logging.getLogger('LabJournal')
logging.basicConfig(level=logging.DEBUG)

In [2]:
import sys
sys.path.append("../") # need this if labjournal is not installed

from simdb.utils.fileFinder import find_files
from simdb.databaseModel import *

In [3]:
SIMS = '/home/andrejb/Research/SIMS'

In [4]:
logger.info('create_database:create_database: FIND FILES')
ERRORS=False
WARNINGS=False

INFO:LabJournal:create_database:create_database: FIND FILES


In [5]:
%%time
SIM_IDS=[]
PATHS=[]
METAS=[]

for fname in find_files(pattern = 'topol.tpr', path = SIMS, dir_ignore = ['data']):
    path =  os.path.dirname(fname)
    sim_id = os.path.basename(path)
    meta = pd.Series.from_csv(os.path.join(path, "meta.csv"))
    SIM_IDS.append(sim_id)
    PATHS.append(path)
    METAS.append(meta)

INFO:Simulation_database:FileFinder: pattern: topol.tpr
INFO:Simulation_database:FileFinder: path: /home/andrejb/Research/SIMS
INFO:Simulation_database:FileFinder: dir_ignore: ['data']
/usr/local/lib/python2.7/dist-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


CPU times: user 788 ms, sys: 188 ms, total: 976 ms
Wall time: 3.33 s


In [6]:
db = 'andrej_raw.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)

INFO:LabJournal:create_database:create_database: removed old file: andrej_raw.db


In [7]:
%%time
for i, entry_id in enumerate(SIM_IDS):
    meta = METAS[i]
    try:
        description = meta['note']
    except KeyError:
        description = ""
    sim = Main(
        entry_id    = entry_id,
        path        = PATHS[i],
        sim_type    = "GROMACS",
        description = description,
        owner       = "andrejb"
    )
    session.add(sim)
    session.flush()
    
    for k, v in meta.iteritems():
        if k != 'note':
            key = Keywords(
                main_id = sim.id,
                name = k,
                value = v
            )
            session.add(key)
session.commit()
session.close()

InvalidRequestError: When initializing mapper Mapper|Main|main, expression 'MetaGroups' failed to locate a name ("name 'MetaGroups' is not defined"). If this is a class name, consider adding this relationship() to the <class 'simdb.databaseModel.Main'> class after both dependent classes have been defined.

In [ ]:
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.query(Main).all()

In [ ]:
session.close()